#### __PrePreProcessing:__
Go over all participants<br>
for each part<br>
-   load 000 folder to Pupil Player<br>
-   cut video<br>
-   cut to frames<br>
-   find first appearince frame of 1st/2nd image
-   Add gaze csv starting time of said image. 
-   add to event.csv?

#### __PreProcessing:__
Go over participants<br>
for each part<br>
-   read event file<br> 
-   read time column<br>
-   clac deltas<br>
-   add gaze csv starting time to all deltas<br>
-   for each time:<br>
    -  go to csv, grab line with closest time <br>
    -  add frame number to output array<br>
    -  add time stamp to output array<br>

The table for each participant:<br>
key:    image number<br>
value:  list of norm_XY points<br> 
-  starts: calced start frame + 10. 
-  ends: starts + 35

In [75]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import pandas as pd
import csv


In [80]:
def get_csv_column(csv_path, column_index, skip_lines = 0):

    col_vals = []

    with open(csv_path, mode='r', newline='') as csv_file:
        
        csv_reader = csv.reader(csv_file)
        
        for _ in range(skip_lines):
            next(csv_reader)

        col_vals = [row[column_index] for row in csv_reader]
        return col_vals


def get_starting_frame(events_csv):
    with open(events_csv, mode='r', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        return next(csv_reader)[2]


def get_images_order(events_csv):
    return get_csv_column(events_csv, 2, 2)


def get_events_timestamps(events_csv):
    return get_csv_column(events_csv, 0, 2)


def calc_deltas(timestamps, factor = 0.0000001):
    start_ts = int(timestamps[0]) * factor
    return [(float(ts) * factor - start_ts) for ts in timestamps]


def get_time_from_frame(gaze_csv, starting_frame):
    gaze = pd.read_csv(gaze_csv)
    frame_idx = int(starting_frame)
    arr_size = 0

    while (0 == arr_size):
        gaze_points = gaze[gaze["world_index"] == frame_idx]
        gaze_points = gaze_points.sort_values(by="gaze_timestamp")
        arr_size = len(gaze_points)
        frame_idx+=1
    
    return gaze_points["gaze_timestamp"].to_numpy()[0]


def calc_gaze_timestamps(events_csv, gaze_csv):

    starting_frame = get_starting_frame(events_csv)
    starting_time = get_time_from_frame(gaze_csv, starting_frame)
    events_timestamps = get_events_timestamps(events_csv)
    deltas = calc_deltas(events_timestamps)

    return [(starting_time + delta) for delta in deltas]


def get_frame_from_time(gaze_csv, timestamp):

    gaze = pd.read_csv(gaze_csv)
    
    gaze['gaze_timestamp'] = gaze['gaze_timestamp'].astype(float)
    # Find the index of the row with the closest timestamp.
    closest_idx = (gaze['gaze_timestamp'] - timestamp).abs().idxmin()
    
    # Return the row as a Series.
    return gaze.loc[closest_idx]


def get_frames_from_timestamps(gaze_csv, timestamps):
    return [get_frame_from_time(gaze_csv, ts)["world_index"] for ts in timestamps]


def write_frames_csv(frames_csv, timestamps, frames, images_order, title=['timestamp', 'frame', 'image']):

    with open(frames_csv, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        # Write the title (column headers) to the CSV file
        writer.writerow(title) 
        
        # Assuming all three lists are of the same length, 
        for timestamp, frame, image in zip(timestamps, frames, images_order):
            writer.writerow([timestamp, frame, image])


In [81]:
# Copy frames.csv template to all folders

parts_root = "parts"
parts_dirs = [part_dir for part_dir in os.listdir(parts_root) if os.path.isdir(os.path.join(parts_root, part_dir))]

for part_dir in parts_dirs:

    part_dir_full = os.path.join(parts_root, part_dir)
    events_csv = os.path.join(part_dir_full, "Events.csv")
    gaze_csv = os.path.join(part_dir_full, "gaze_positions.csv")
    frames_csv = os.path.join(part_dir_full, "Frames.csv")
    print(gaze_csv)
    if not os.path.exists(frames_csv):
        Path(frames_csv).touch()

    gaze_timestamps = calc_gaze_timestamps(events_csv, gaze_csv)
    frames = get_frames_from_timestamps(gaze_csv, gaze_timestamps)   
    image_order = get_images_order(events_csv) 
    write_frames_csv(frames_csv, gaze_timestamps, frames, image_order)


parts\ASH\gaze_positions.csv
parts\DI\gaze_positions.csv
parts\MA\gaze_positions.csv
parts\SHM\gaze_positions.csv
parts\SHO\gaze_positions.csv
parts\THC\gaze_positions.csv
parts\YK\gaze_positions.csv
